In [ ]:
pip install pyautogen
pip install xgboost

In [ ]:
import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent

* Downloaded **Mistral-7B-Instruct-v0.1-GGUF** LLM via LM Studio from Hugging Face.

* Deployed LLM model on local server.

## Using ConversableAgent

In [ ]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}


Bill= ConversableAgent(
    name="assistant",
    llm_config=mistral,
    system_message="You are Bill and you are a comedian."
)


Burr= ConversableAgent(
    name="assistant",
    llm_config=mistral,
    system_message="You are Burr and you are a part of two person comedy show."
)


chat_result = Bill.initiate_chat(Burr,message="Burr, tell me a joke",max_turns=2)


assistant (to assistant):

Burr, tell me a joke

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 Sure thing! Here's one for you:

Why did the tomato turn red?

Because it saw the salad dressing!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 That's a classic, Timmy. Do you have any other good ones up your sleeve?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to assistant):

 Of course I do! How about this one:

Why was the math book sad?

Because it had too many problems!

--------------------------------------------------------------------------------


In [ ]:
##############################################################################################################################################################

## AssistantAgent and UserProxyAgent

In [ ]:
assistant = AssistantAgent("assistant", llm_config={"config_list": mistral['config_list']})
user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended
user_proxy.initiate_chat(assistant, message="Tell me a Joke.")

user_proxy (to assistant):

Tell me a Joke.

--------------------------------------------------------------------------------
assistant (to user_proxy):

 Sure! Here's a joke for you:

Why was the math book sad? Because it had too many problems. 

I hope you found that joke amusing! Let me know if you need anything else.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Tell me a Joke.', 'role': 'assistant'}, {'content': " Sure! Here's a joke for you:\n\nWhy was the math book sad? Because it had too many problems. \n\nI hope you found that joke amusing! Let me know if you need anything else.", 'role': 'user'}], summary=" Sure! Here's a joke for you:\n\nWhy was the math book sad? Because it had too many problems. \n\nI hope you found that joke amusing! Let me know if you need anything else.", cost={'usage_including_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf': {'cost': 0, 'prompt_tokens': 498, 'completion_tokens': 47, 'total_tokens': 545}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf': {'cost': 0, 'prompt_tokens': 498, 'completion_tokens': 47, 'total_tokens': 545}}}, human_input=['exit'])

In [ ]:
##############################################################################################################################################################

## Groupchat (Planner, Admin, Analyst, Engineer)


Engineer: write code

Analyst: Analyses and gets insights

Planner: Suggests plan

Executor: Execute the code

In [ ]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}

In [ ]:
# Engineer
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=mistral,
    system_message="""Engineer. You follow an approved plan. You write python code to solve tasks. The user can't modify your code. Do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
Don't include multiple code blocks in one response. Do not ask others to copy and paste the result.
""",
)

# Scientist
scientist = autogen.AssistantAgent(
    name="Analyst",
    llm_config=mistral,
    system_message="""Analyst. You follow an approved plan. You are able to draw conclusion from codes and graphs. You don't write code.""",
)

# Admin
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)

#Planner
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin, until admin approval.
The plan may involve an engineer who can write code and an Analyst who doesn't write code.
Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a Analyst.
""",
    llm_config=mistral,
)

# Executor
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },
)

# Groupchat
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor], messages=[], max_round=30
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=mistral)

In [4]:
user_proxy.initiate_chat(
    manager,
    message="""
Should an investor invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks. If stock prices are increasing for next 5 days, invest in Apple stocks.
""",
)

Admin (to chat_manager):


Should an investor invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks. If stock prices are increasing for next 5 days, invest in Apple stocks.


--------------------------------------------------------------------------------
Planner (to chat_manager):

## Plan: Apple Stock Investment Recommendation

**Goal:** Determine if an investor should invest in Apple stocks based on a 5-day time series forecast.

**Roles:**

* **Analyst:** Responsible for data gathering, analysis, and interpretation.
* **Engineer:** Responsible for building and executing the forecasting model.

**Steps:**

1. **Data Gathering (Analyst):**
    * Obtain historical Apple stock price data for a suitable timeframe (e.g., past 5 years).
    * Ensure data quality and consistency.
2. **Model Selection (Engineer):**
    * Select an appropriate time series forecasting model (e.g., ARIMA, LSTM, Prophet).
    * Define model parameters based on data characteristi

ChatResult(chat_id=None, chat_history=[{'content': '\nShould an investor invest in Apple stocks? Suggest based on time series forecast analysis of Apple stocks. If stock prices are increasing for next 5 days, invest in Apple stocks.\n', 'role': 'assistant'}, {'content': "## Plan: Apple Stock Investment Recommendation\n\n**Goal:** Determine if an investor should invest in Apple stocks based on a 5-day time series forecast.\n\n**Roles:**\n\n* **Analyst:** Responsible for data gathering, analysis, and interpretation.\n* **Engineer:** Responsible for building and executing the forecasting model.\n\n**Steps:**\n\n1. **Data Gathering (Analyst):**\n    * Obtain historical Apple stock price data for a suitable timeframe (e.g., past 5 years).\n    * Ensure data quality and consistency.\n2. **Model Selection (Engineer):**\n    * Select an appropriate time series forecasting model (e.g., ARIMA, LSTM, Prophet).\n    * Define model parameters based on data characteristics.\n3. **Model Training (Eng

In [ ]:
##############################################################################################################################################################

## RAG Implementation in Autogen

In [9]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [ ]:
mistral={
"config_list":[{
    "model":"TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q2_K.gguf",
    "base_url":"http://localhost:1234/v1",
    "api_key":"lm-studio"
},],
"cache_seed":None
}


task='qa'

In [11]:
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=config_list_gemini[0],
    code_execution_config=False
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "qa",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
    },
    code_execution_config=False,
)

In [12]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, message=ragproxyagent.message_generator, problem="What is autogen? Write 300 words essay on it.")

Trying to create collection.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:autogen.retrieve_utils:Found 2 chunks.


doc_ids:  [['doc_0', 'doc_1']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
ragproxyagent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is autogen? Write 300 words essay on it.

Context is: <a name="readme-top"></a>

[![PyPI version](https://badge.fury.io/py/pyautogen.svg)](https://badge.fury.io/py/pyautogen)
[![Build](https://github.com/microsoft/autogen/actions/workflows/python-package.yml/badge.svg)](https://github.com/microsoft/autogen/actions/workflows/python-package.yml)
![Python Version](https://img.shields.io/badge/3.8%20%7C%203.9%20%7C%203.10%20%7C%203.11%20%7C%203.12-blue)
[![Downloads](https://static.pepy.tech/badge/pyautogen/week)](https://pepy.tech/project/pyautogen)
[![Di

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nYou must give as short an answer as possible.\n\nUser\'s question is: What is autogen? Write 300 words essay on it.\n\nContext is: <a name="readme-top"></a>\n\n[![PyPI version](https://badge.fury.io/py/pyautogen.svg)](https://badge.fury.io/py/pyautogen)\n[![Build](https://github.com/microsoft/autogen/actions/workflows/python-package.yml/badge.svg)](https://github.com/microsoft/autogen/actions/workflows/python-package.yml)\n![Python Version](https://img.shields.io/badge/3.8%20%7C%203.9%20%7C%203.10%20%7C%203.11%20%7C%203.12-blue)\n[![Downloads](https://static.pepy.tech/badge/pyautogen/week)](https://pepy.tech/project/pyautogen)\n[![Discord](https://img.shields.io/discord/1153072414184452236

task = 'code'

In [13]:
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=config_list_gemini[0],
    code_execution_config=False
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/autogen/main/README.md",
    },
    code_execution_config=False,
)

In [15]:
assistant.reset()
ragproxyagent.initiate_chat(assistant, message="Write a code for Dijkstra's algorithm and explain the code.")

ragproxyagent (to assistant):

Write a code for Dijkstra's algorithm and explain the code.

--------------------------------------------------------------------------------
assistant (to ragproxyagent):

```python
import sys

def dijkstra(graph, start_node):
    """
    Calculates the shortest distances from a starting node to all other nodes in a graph using Dijkstra's algorithm.

    Args:
        graph: A dictionary representing the graph, where keys are nodes and values are dictionaries of neighboring nodes with associated edge weights.
        start_node: The starting node for the shortest path calculations.

    Returns:
        A dictionary of shortest distances from the start node to all other nodes.
    """

    # Initialize distances to infinity for all nodes except the start node
    distances = {node: float('inf') for node in graph}
    distances[start_node] = 0

    # Initialize a set of unvisited nodes
    unvisited = set(graph)

    # Initialize a dictionary to store the

ChatResult(chat_id=None, chat_history=[{'content': "Write a code for Dijkstra's algorithm and explain the code.", 'role': 'assistant'}, {'content': '```python\nimport sys\n\ndef dijkstra(graph, start_node):\n    """\n    Calculates the shortest distances from a starting node to all other nodes in a graph using Dijkstra\'s algorithm.\n\n    Args:\n        graph: A dictionary representing the graph, where keys are nodes and values are dictionaries of neighboring nodes with associated edge weights.\n        start_node: The starting node for the shortest path calculations.\n\n    Returns:\n        A dictionary of shortest distances from the start node to all other nodes.\n    """\n\n    # Initialize distances to infinity for all nodes except the start node\n    distances = {node: float(\'inf\') for node in graph}\n    distances[start_node] = 0\n\n    # Initialize a set of unvisited nodes\n    unvisited = set(graph)\n\n    # Initialize a dictionary to store the predecessor nodes for each no